In [1]:
import gym      # environment 라이브러리
import torch    # pytorch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Hyperparameters
learning_rate = 0.0005
gamma = 0.98

In [2]:
# pytorch의 nn.Module 클래스를 상속해서 만든다. 
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        # 에피소드 완료되기 전까지 임시로 데이터를 저장할 변수
        self.data = []

        # state가 4개이기 때문에 feature vector는 4차원이다. 은닉층은 128차원으로 잡았다.
        # nn.Linear()는 fully connected이다. 즉, 4차원에서 128차원으로 가는 선형변환이다.
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    # 정방향 전달하는 모든 호출에서 수행할 계산을 정의한다.
    def forward(self, x):
        # 4차원 feature vector를 128차원으로 변환하고 ReLU()를 취한 값을 tensor로 return한다.
        x = F.relu(self.fc1(x))
        # 128차원을 2차원으로 바꾸고 softmax()를 취한 값을 tensor로 return한다.
        x = F.softmax(self.fc2(x), dim=0)
        return x

    # 데이터 쌓아두기
    def put_data(self, item):
        self.data.append(item)

    # 네트워크 학습시키기
    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        # 거꾸로 보는 이유는 return을 쉽게 계산하기 위해서이다. 
        # 예를 들어, 에피소드가 100스텝이라고 하자. 
        # 99스텝에서 reward를 구하면 이게 return이다.
        # 98스텝에서 reward를 구하면, 99스텝에서 계산한 return에 감마를 곱하고 reward를 더해서 return을 구할 수 있다.
        for r, prob in self.data[::-1]:
            # return을 cumulative하게 계산해 나간다.
            R = r + gamma * R
            # loss는 -log(pi) * R 로 정의한다.
            loss = -torch.log(prob) * R
            # autograd가 backpropagation을 자동으로 처리한다.
            # model의 parameter에 대한 loss의 gradient를 계산한다.
            loss.backward()
        # single optimization step을 수행하여 parameter를 update함.
        self.optimizer.step()
        # 이미 학습한 데이터를 제거한다.
        self.data = []

In [3]:
def main():
    # environment를 생성한다. 
    env = gym.make('CartPole-v1')
    # Policy 클래스의 인스턴스를 생성한다.
    pi = Policy()
    score = 0.0
    print_interval = 20

    # episode를 만 번 시뮬레이션 한다.
    for n_epi in range(10000):
        # env를 처음 상태로 초기화함과 동시에 observation 결과, 즉 state를 돌려준다.
        # CartPole의 경우 state는 네 개의 실수로 구성되어 있다. 
        # CartPole 환경의 자세한 사항은 https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py 에서 확인.
        s = env.reset()
        done = False

        while not done:
            # numpy.ndarray인 s를 tensor로 바꿔서 policy input으로 넣어준다. 그럼 output으로 확률분포를 얻는다. 
            # CartPole의 경우 action이 2개이다. 왼쪽으로 밀기, 오른쪽으로 밀기
            # 예를 들어 (왼쪽으로 밀 확률 0.8, 오른쪽으로 밀 확률 0.2) 와 같이 확률 분포가 주어진다.
            prob = pi(torch.from_numpy(s).float())
            # policy가 stochastic policy이므로 sampling을 해야 한다.
            # pytorch의 Categorical은 확률분포 모델이다.
            m = Categorical(prob)
            # 이 모델에서 sample을 호출하면 확률분포에 맞게 action을 tensor로 뽑아준다.
            a = m.sample()
            # env.step에 action을 주면 그 결과의 observation을 얻는다. =state transition
            # a.item()은 tensor에서 scalar를 추출하기 위해 호출한 것이다.
            s_prime, r, done, info = env.step(a.item())
            # REINFORCE 알고리즘은 return이 필요하기 때문에 에피소드가 끝나야 학습할 수 있다.
            # for 문을 돌면서 얻는 경험을 policy에 쌓아두기만 한다.
            # (현재 reward, 현재 action을 선택할 확률)
            pi.put_data((r, prob[a]))
            s = s_prime
            # score는 reward의 누적인데, reward는 매 스텝을 버틸 때마다 +1이 주어진다.
            score += r

        # 에피소드가 종료되었으므로 학습시킨다.
        pi.train_net()

        if n_epi % print_interval == 0 and n_epi != 0:
            # 20 에피소드 평균 score를 출력함.
            print(f"# of epi: {n_epi}, avg score: {score/print_interval}")
            score = 0.0
    env.close()

In [4]:
if __name__ == '__main__':
    main()

# of epi: 20, avg score: 21.9
# of epi: 40, avg score: 21.8
# of epi: 60, avg score: 28.9
# of epi: 80, avg score: 27.8
# of epi: 100, avg score: 36.45
# of epi: 120, avg score: 39.05
# of epi: 140, avg score: 26.55
# of epi: 160, avg score: 37.6
# of epi: 180, avg score: 44.95
# of epi: 200, avg score: 61.85
# of epi: 220, avg score: 43.65
# of epi: 240, avg score: 42.2
# of epi: 260, avg score: 68.25
# of epi: 280, avg score: 53.9
# of epi: 300, avg score: 63.1
# of epi: 320, avg score: 66.35
# of epi: 340, avg score: 80.75
# of epi: 360, avg score: 67.7
# of epi: 380, avg score: 68.0
# of epi: 400, avg score: 102.6
# of epi: 420, avg score: 91.55
# of epi: 440, avg score: 129.75
# of epi: 460, avg score: 120.45
# of epi: 480, avg score: 151.2
# of epi: 500, avg score: 156.4
# of epi: 520, avg score: 143.5
# of epi: 540, avg score: 195.35
# of epi: 560, avg score: 160.95
# of epi: 580, avg score: 212.1
# of epi: 600, avg score: 195.95
# of epi: 620, avg score: 230.6
# of epi: 640, av

KeyboardInterrupt: 